In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-joj1pcxy
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-joj1pcxy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=213d290f802b04ffab32d6894e75694bfdf512e825139868f298c0b342cf3760
  Stored in directory: /tmp/pip-ephem-wheel-cache-4t6nnhc3/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <cuda_runtime_api.h>
#include <algorithm>
#include <cassert>
#include <cstdlib>
#include <time.h>
#include <iostream>
#include "device_launch_parameters.h"
using namespace std;

#define MAX_MASK_WIDTH 5
__constant__ int M[MAX_MASK_WIDTH];

__global__ void convolution_2D_basic_kernel(int *N, int * M, int *P, int m_width, int a_width, int a_height){
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int id = row*a_width + col;
    extern __shared__ int sdata[];
    sdata[id] = N[id];
    __syncthreads();

    int val=0;
    int N_start_col = col - (m_width/2);
    int N_start_row = row - (m_width/2);

    for (int i=0; i<m_width; i++){
        for (int j=0; j<m_width; j++){
            int curRow = N_start_row+i; 
            int curCol = N_start_col+j; 

            if (curRow >= 0 && curRow < a_height && curCol >= 0 && curCol < a_width){
                val += sdata[curRow*a_width+curCol]*M[i*m_width+j];
            }
        }
    }
    

    P[row*a_width+col] = val;
}

int main(){ 
  int ARRAY_SIZE = 25;
  int ARRAY_WIDTH = 5;
  int MASK_WIDTH = 3;
  int MASK_SIZE = 9;

  int h_N[] = {2, 2, 2, 2, 2,
               2, 2, 2, 2, 2,         
               2, 2, 2, 2, 2,
               2, 2, 2, 2, 2,
               2, 2, 2, 2, 2};      

  int h_M[] = {1, 1, 1,
               1, 1, 1,
               1, 1, 1};

  int *h_P;
  int* d_N, * d_M,  * d_P;
  cudaMallocHost(&h_P, ARRAY_SIZE * sizeof(int));

  cudaMalloc(&d_N, ARRAY_SIZE * sizeof(int));
  cudaMalloc(&d_M, MASK_SIZE*sizeof(int));
  cudaMalloc(&d_P, ARRAY_SIZE * sizeof(int));

  cudaMemcpy(d_N, h_N, ARRAY_SIZE * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_M, h_M, MASK_SIZE * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpyToSymbol(M, h_M, MASK_WIDTH * sizeof(int));

  dim3 GridSize(1,1,1);
  dim3 BlockSize(5,5,1);
  convolution_2D_basic_kernel<<<GridSize, BlockSize, 25*sizeof(int)>>>(d_N, d_M, d_P, MASK_WIDTH, ARRAY_WIDTH, ARRAY_WIDTH);

  cudaMemcpy(h_P, d_P, ARRAY_SIZE * sizeof(int), cudaMemcpyDeviceToHost);  
  cudaDeviceSynchronize();
  cout << "OUTPUT:";
  for (int i=0; i<ARRAY_SIZE; i++){
    if (i%ARRAY_WIDTH==0) cout<<endl;
    cout << h_P[i] << " ";
  }

  return 0;
}

OUTPUT:
8 12 12 12 8 
12 18 18 18 12 
12 18 18 18 12 
12 18 18 18 12 
8 12 12 12 8 
